code used to generate the HMC visualisation 

In [2]:
import numpy as np
import scipy.stats
from matplotlib import pyplot as plt
!pip install celluloid
!pip install ffmpeg
from celluloid import Camera
import ffmpeg

  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6084 sha256=d7a2693d0704db329a9051aeac052e1bf02de513139258e0727da9a3ef53ddf2
  Stored in directory: /root/.cache/pip/wheels/64/80/6e/caa3e16deb0267c3cbfd36862058a724144e19fdb9eb03af0f
Successfully built ffmpeg


In [3]:
def U(x):
  val = 0.5*(x**2) + np.log(np.sqrt(2*np.pi))
  return val
def gradU(x):
  return x  

def HMC(U, gradU, epsilons, L, currentq, n):
  samples = np.zeros((n, len(currentq)))
  accept = np.zeros(n)
  trajq = np.zeros(n*L)
  trajp = np.zeros((n, len(currentq)))
  for j in range(0, n):
    q = np.copy(currentq)
    p = np.random.normal(size=len(currentq)) 
    trajp[j] = p
    currentp = np.copy(p)
    "perform L leapfrog steps"
    p = p - epsilons*gradU(q)/2
    for i in range(0, L):
      q = q + epsilons*p
      trajq[L*j + i] = q 
      if i != L-1:
        p = p - epsilons*gradU(q)
    p = p - epsilons*gradU(q)/2
    p = -1*p
    "calculate inital and proposed potential and kinetic energies"
    currentU = U(currentq)
    proposedU = U(q)
    currentK = sum(0.5*(currentp**2))
    proposedK = sum(0.5*(p**2))
    "Metropolis acceptance rule"
    if np.random.uniform(size=1) < np.exp(currentU+currentK - proposedU - proposedK):
      currentq = q
      accept[j] = 1 
    samples[j, ] = currentq   
  return [samples, accept, trajq, trajp] 

n=10000
L=73
[results, acc, trajectoryq, trajectoryp] = HMC(U, gradU, 0.15, L, np.array([0]), n)

In [ ]:
fig, ax1 = plt.subplots()
ax1.set_xlabel('Position')
ax1.set_ylabel('Potential energy (- log of target)', color='blue')
ax1.set_ylim(-0.2,9)
ax2 = ax1.twinx()  
ax2.set_ylabel('N(0,1) Density (target)', color='darkorange')
ax2.set_ylim(-0.03,0.45)
camera = Camera(fig)
t = np.arange(-4, 4, 0.05)
yy = U(t)
yy2 = scipy.stats.norm.pdf(t)

for j in list(range(0, 4))+list(range(97, 101))+list(range(9997,10000)):
  for i in trajectoryq[j*L:(j+1)*L-1]:
    ax1.plot(t, yy, color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax2.plot(t, yy2, color='darkorange')
    ax2.tick_params(axis='y', labelcolor='darkorange') 
    ax1.scatter(results[0:j], np.full(j, -0.02), c='green', s=30,alpha=0.25)
    if j>=2:
      dens = scipy.stats.gaussian_kde(np.concatenate(results[0:j]))
      ax2.plot(t, dens(t), c='green')
    ax2.text(-3.5,0.47,'Iteration'.ljust(10)+str(j+1)+','.ljust(2)+'momentum='+str(round(trajectoryp[j][0],2)), fontsize=15)
    ax1.scatter(i,U(i), c='black', s=80)
    camera.snap()
  for i in range(0,30):
    ax1.plot(t, yy, color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax2.plot(t, yy2, color='darkorange')
    ax2.tick_params(axis='y', labelcolor='darkorange')
    ax1.scatter(results[0:j], np.full(j, -0.02), c='green', s=30,alpha=0.25)
    if j>=2:
      dens = scipy.stats.gaussian_kde(np.concatenate(results[0:j]))
      ax2.plot(t, dens(t), c='green')
    ax2.text(-3.5,0.47,'Iteration'.ljust(10)+str(j+1)+','.ljust(2)+'momentum='+str(round(trajectoryp[j][0],2)), fontsize=15)
    ax1.scatter(trajectoryq[(j+1)*L-1],U(trajectoryq[(j+1)*L-1]), c='green', s=80)
    camera.snap()

ax1.plot(t, yy, color='blue')
ax1.tick_params(axis='y', labelcolor='blue')
ax2.plot(t, yy2, color='darkorange')
ax2.tick_params(axis='y', labelcolor='darkorange')
ax1.scatter(results, np.full(n, -0.02), c='green', s=30,alpha=0.25)
dens = scipy.stats.gaussian_kde(np.concatenate(results))
ax2.plot(t, dens(t), c='green')
ax2.text(-3.5,0.47,'Iteration'.ljust(10)+str(n)+','.ljust(2)+'momentum='+str(round(trajectoryp[n-1][0],2)), fontsize=15)
ax1.scatter(trajectoryq[n*L-1],U(trajectoryq[n*L-1]), c='black', s=80)
camera.snap()

animation = camera.animate(interval = 30)  
animation.save('HMCvis.mp4')